In [1]:
import os
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm.auto import tqdm

/users/PAS2985/tingle9/awsenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FootprintPatchDataset(Dataset):
    """
    root_dir/
      class_name0/
        *.jpg, *.png ...
      class_name1/
        ...
    """
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}
        self.idx_to_class = []

        for class_idx, class_name in enumerate(sorted(os.listdir(self.root_dir))):
            class_path = self.root_dir / class_name
            if not class_path.is_dir():
                continue

            self.class_to_idx[class_name] = class_idx
            self.idx_to_class.append(class_name)

            for fname in os.listdir(class_path):
                if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.image_paths.append(class_path / fname)
                    self.labels.append(class_idx)

        print(f"Loaded {len(self.image_paths)} images "
              f"from {self.root_dir}, {len(self.idx_to_class)} classes.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [3]:
class FootprintEncoder(nn.Module):
    def __init__(self, feature_dim=256):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(256, feature_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class CNNClassifier(nn.Module):
    """
    CNNEncoder + Linear classifier (supervised)
    """
    def __init__(self, num_classes, feature_dim=256):
        super().__init__()
        self.encoder = FootprintEncoder(feature_dim=feature_dim)
        self.fc = nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        feats = self.encoder(x)
        logits = self.fc(feats)
        return logits

class BackboneEncoder(nn.Module):
    def __init__(self, backbone="resnet50", num_classes=117, pretrained=True):
        super().__init__()
        self.backbone_name = backbone

        if backbone == "resnet50":
            m = models.resnet50(
                weights=models.ResNet50_Weights.DEFAULT if pretrained else None
            )
            in_dim = m.fc.in_features
            m.fc = nn.Linear(in_dim, num_classes)
            self.model = m

        elif backbone == "vgg16":
            m = models.vgg16(
                weights=models.VGG16_Weights.DEFAULT if pretrained else None
            )
           
            in_dim = m.classifier[-1].in_features
            m.classifier[-1] = nn.Linear(in_dim, num_classes)
            self.model = m

        elif backbone == "vit_b_16":
            m = models.vit_b_16(
                weights=models.ViT_B_16_Weights.DEFAULT if pretrained else None
            )
            
            if isinstance(m.heads, nn.Sequential):
                # heads = Sequential(Linear(768, 1000))
                in_dim = m.heads[0].in_features
                m.heads = nn.Linear(in_dim, num_classes)
            else:
                # heads = Linear(768, 1000)
                in_dim = m.heads.in_features
                m.heads = nn.Linear(in_dim, num_classes)
            self.model = m

        else:
            raise ValueError(f"Unknown backbone: {backbone}")

    def forward(self, x):
        return self.model(x)

In [4]:
def accuracy(output, target, topk=(1,)):
    """
    output: (B, C) logits
    target: (B,)
    topk: (1,5) tuple

    return: [top1, top5]
    """
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, dim=1, largest=True, sorted=True)  # (B, maxk)
    pred = pred.t()                                                # (maxk, B)
    correct = pred.eq(target.view(1, -1).expand_as(pred))          # (maxk, B)

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append((correct_k / batch_size).item())
    return res


def train_one_epoch(model, loader, optimizer, device, criterion):
    model.train()
    total_loss = 0.0
    total_samples = 0
    total_top1 = 0.0
    total_top5 = 0.0

    for images, labels in tqdm(loader, desc="Train", leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        logits = model(images)               # (B, num_classes)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        bsz = labels.size(0)
        total_loss += loss.item() * bsz
        total_samples += bsz

        top1, top5 = accuracy(logits, labels, topk=(1,5))
        total_top1 += top1 * bsz
        total_top5 += top5 * bsz

    avg_loss = total_loss / total_samples
    avg_top1 = total_top1 / total_samples
    avg_top5 = total_top5 / total_samples

    return avg_loss, avg_top1, avg_top5

def evaluate(model, loader, device, criterion=None):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_top1 = 0.0
    total_top5 = 0.0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Eval", leave=False):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            logits = model(images)

            if criterion is not None:
                loss = criterion(logits, labels)
                total_loss += loss.item() * labels.size(0)

            bsz = labels.size(0)
            total_samples += bsz

            top1, top5 = accuracy(logits, labels, topk=(1,5))
            total_top1 += top1 * bsz
            total_top5 += top5 * bsz

    avg_loss = total_loss / total_samples if criterion is not None else None
    avg_top1 = total_top1 / total_samples
    avg_top5 = total_top5 / total_samples
    return avg_loss, avg_top1, avg_top5

In [5]:
def main():
    PATCH_ROOT = Path("/users/PAS2985/tingle9/dataset/footprint_patches")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    backbones = ["cnn", "resnet50", "vgg16", "vit_b_16"]

    num_epochs = 50
    batch_size = 64
    num_workers = 8

    IMAGENET_MEAN = (0.485, 0.456, 0.406)
    IMAGENET_STD  = (0.229, 0.224, 0.225)

    for backbone in backbones:
        print(f"Supervised training backbone = {backbone}")

        if backbone == "vit_b_16":
            image_size = 224
        else:
            image_size = 128

        if backbone == "cnn":
            train_transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
            ])
            
            eval_transform = train_transform

        else:
            train_transform = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ])

            eval_transform = train_transform

        # ----- Dataset / DataLoader -----
        train_ds = FootprintPatchDataset(PATCH_ROOT / "train", transform=train_transform)
        val_ds   = FootprintPatchDataset(PATCH_ROOT / "valid", transform=eval_transform)
        test_ds  = FootprintPatchDataset(PATCH_ROOT / "test",  transform=eval_transform)

        num_classes = len(train_ds.idx_to_class)
        print("Num classes:", num_classes)

        train_loader = DataLoader(
            train_ds,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=True,
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=True,
        )
        test_loader = DataLoader(
            test_ds,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True,
            persistent_workers=True,
        )

        if backbone == "cnn":
            model = CNNClassifier(
                num_classes=num_classes,
                feature_dim=256
            ).to(device)
            lr = 1e-3
        else:
            model = BackboneEncoder(
                backbone=backbone,
                num_classes=num_classes,
                pretrained=True
            ).to(device)
            lr = 1e-4

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=1e-4
        )
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=30,
            gamma=0.1
        )

        best_val_top1 = 0.0
        best_ckpt_path = f"supervised_{backbone}_best.pth"

        # ----- Training loop -----
        for epoch in range(num_epochs):
            print(f"\n[Backbone={backbone}] Epoch {epoch+1}/{num_epochs}")

            train_loss, train_top1, train_top5 = train_one_epoch(
                model, train_loader, optimizer, device, criterion
            )
            val_loss, val_top1, val_top5 = evaluate(
                model, val_loader, device, criterion
            )

            scheduler.step()

            print(
                f"  Train: loss={train_loss:.4f}, "
                f"top1={train_top1*100:.2f}%, top5={train_top5*100:.2f}%"
            )
            print(
                f"  Val  : loss={val_loss:.4f}, "
                f"top1={val_top1*100:.2f}%, top5={val_top5*100:.2f}%"
            )

            if val_top1 > best_val_top1:
                best_val_top1 = val_top1
                torch.save(model.state_dict(), best_ckpt_path)

        print(f"\n[Backbone={backbone}] evaluating on TEST...")
        model.load_state_dict(torch.load(best_ckpt_path, map_location=device))
        test_loss, test_top1, test_top5 = evaluate(
            model, test_loader, device, criterion
        )
        print(
            f"[Backbone={backbone}] TEST: "
            f"loss={test_loss:.4f}, "
            f"top1={test_top1*100:.2f}%, top5={test_top5*100:.2f}%"
        )

main()

Using device: cuda
Supervised training backbone = cnn
Loaded 12575 images from /users/PAS2985/tingle9/dataset/footprint_patches/train, 117 classes.
Loaded 1882 images from /users/PAS2985/tingle9/dataset/footprint_patches/valid, 117 classes.
Loaded 3831 images from /users/PAS2985/tingle9/dataset/footprint_patches/test, 117 classes.
Num classes: 117

[Backbone=cnn] Epoch 1/50


  Train: loss=4.1940, top1=6.40%, top5=21.28%
  Val  : loss=4.2964, top1=4.20%, top5=19.61%

[Backbone=cnn] Epoch 2/50


  Train: loss=4.0043, top1=8.44%, top5=25.66%
  Val  : loss=4.2358, top1=5.47%, top5=21.04%

[Backbone=cnn] Epoch 3/50


  Train: loss=3.9003, top1=9.58%, top5=28.29%
  Val  : loss=4.1663, top1=6.91%, top5=24.18%

[Backbone=cnn] Epoch 4/50


  Train: loss=3.8125, top1=10.43%, top5=31.24%
  Val  : loss=4.4231, top1=4.78%, top5=19.34%

[Backbone=cnn] Epoch 5/50


...

  Train: loss=0.0003, top1=100.00%, top5=100.00%
  Val  : loss=4.3039, top1=32.09%, top5=60.89%

[Backbone=vit_b_16] Epoch 48/50


  Train: loss=0.0003, top1=100.00%, top5=100.00%
  Val  : loss=4.3166, top1=32.20%, top5=60.84%

[Backbone=vit_b_16] Epoch 49/50


  Train: loss=0.0003, top1=100.00%, top5=100.00%
  Val  : loss=4.3298, top1=32.09%, top5=60.95%

[Backbone=vit_b_16] Epoch 50/50


  Train: loss=0.0003, top1=100.00%, top5=100.00%
  Val  : loss=4.3426, top1=32.09%, top5=61.00%

[Backbone=vit_b_16] evaluating on TEST...


[Backbone=vit_b_16] TEST: loss=4.4191, top1=30.83%, top5=60.38%
